In [48]:
import pennylane as qml
from pennylane import numpy as np
import torch
from torch.autograd import Variable

In [34]:
# create a device to execute the circuit on
dev = qml.device("default.qubit", wires=8)


@qml.qnode(dev, intergace="torch", diff_method="parameter-shift")
def chsh_circuit(params):
    qml.Hadamard(wires=0)
    qml.Hadamard(wires=2)
    qml.Hadamard(wires=4)
    qml.Hadamard(wires=6)
    
    qml.CNOT(wires=[0,1])
    qml.CNOT(wires=[2,3])
    qml.CNOT(wires=[4,5])
    qml.CNOT(wires=[6,7])

    qml.RY(params[0],wires=[0])
    qml.RY(params[2],wires=[1])
    qml.RY(params[0],wires=[2])
    qml.RY(params[3],wires=[3])
    qml.RY(params[1],wires=[4])
    qml.RY(params[2],wires=[5])
    qml.RY(params[1],wires=[6])
    qml.RY(params[3],wires=[7])
    
    return [
        qml.expval(qml.PauliZ(0) @ qml.PauliZ(1)),
        qml.expval(qml.PauliZ(2) @ qml.PauliZ(3)),
        qml.expval(qml.PauliZ(4) @ qml.PauliZ(5)),
        qml.expval(qml.PauliZ(6) @ qml.PauliZ(7)),
    ]

In [36]:
print("Expectation value:", chsh_circuit([0,np.pi/2,np.pi/4,-np.pi/4]))

Expectation value: [ 0.70710678  0.70710678  0.70710678 -0.70710678]


In [37]:
print(chsh_circuit.draw())


 0: ──H──╭C──RY(0)───────╭┤ ⟨Z ⊗ Z⟩ 
 1: ─────╰X──RY(0.785)───╰┤ ⟨Z ⊗ Z⟩ 
 2: ──H──╭C──RY(0)───────╭┤ ⟨Z ⊗ Z⟩ 
 3: ─────╰X──RY(-0.785)──╰┤ ⟨Z ⊗ Z⟩ 
 4: ──H──╭C──RY(1.57)────╭┤ ⟨Z ⊗ Z⟩ 
 5: ─────╰X──RY(0.785)───╰┤ ⟨Z ⊗ Z⟩ 
 6: ──H──╭C──RY(1.57)────╭┤ ⟨Z ⊗ Z⟩ 
 7: ─────╰X──RY(-0.785)──╰┤ ⟨Z ⊗ Z⟩ 



In [58]:
def chsh_cost(exp_vals):
    chsh_score = exp_vals[0] + exp_vals[1] + exp_vals[2] - exp_vals[3]
    return torch.mul(-1,torch.tensor(chsh_score))
    

In [65]:
# number of qubits in the circuit
num_qubits = 8
np.random.seed(42)

# randomly initialize parameters from a normal distribution
params = np.random.normal(0, np.pi, 4)
params = Variable(torch.tensor(params), requires_grad=False)

# set up the optimizer
opt = torch.optim.Adam([params], lr=0.1)

# number of steps in the optimization routine
steps = 20

# the final stage of optimization isn't always the best, so we keep track of
# the best parameters along the way
best_cost = chsh_cost(chsh_circuit(params))
best_params = np.zeros(4)

print("Cost after 0 steps is {:.4f}".format(chsh_cost(circuit(params))))

# optimization begins
for n in range(steps):
    opt.zero_grad()
    loss = chsh_cost(circuit(params))
#     loss.backward()
    opt.step()

    # keeps track of best parameters
    if loss < best_cost:
        best_params = params

    # Keep track of progress every 10 steps
    if n % 10 == 9 or n == steps - 1:
        print("Cost after {} steps is {:.4f}".format(n + 1, loss))

# calculate the Bloch vector of the output state
exp_vals = chsh_circuit(best_params)
opt_score = -1*chsh_cost(chsh_circuit(best_params))


# print results
print("Optimized exp vals = ", exp_vals)
print("Optimal chsh score = ", opt_score)



Cost after 0 steps is 1.3746
Cost after 10 steps is 1.3746
Cost after 20 steps is 1.3746
Optimized exp vals =  [1. 1. 1. 1.]
Optimal chsh score =  tensor(2.0000, dtype=torch.float64)


In [62]:
params = np.random.normal(0, np.pi, 4)
params

tensor([-0.73561452, -0.73556294,  4.96124338,  2.41096731], requires_grad=True)